In [14]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

import pandas as pd
import numpy as np

In [15]:
train_df = pd.read_csv('./data/iris.csv')
np.random.shuffle(train_df.values)

print(train_df.head())

train_x = np.column_stack((train_df.SepalLengthCm.values, train_df.SepalWidthCm.values, train_df.PetalLengthCm.values, train_df.PetalWidthCm.values))

   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa


In [16]:
print(train_df.Species.unique())

species_dict = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

train_df['Species'] = train_df.Species.apply(lambda x: species_dict[x])

print(train_df.Species.unique())

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
[0 1 2]


In [17]:
model = keras.Sequential([
	keras.layers.Dense(64, input_shape=(4,), activation='relu'),
    # keras.layers.Dropout(0.2),									# % of neurons (of previous layer) that will be dropped
    keras.layers.Dense(64, activation='relu'),
	keras.layers.Dense(64, activation='relu'),						# adding layer, because quadratic example is more complex
	keras.layers.Dense(3, activation='sigmoid')])

In [29]:
optimizers = {'Adadelta': None, 'Adagrad': None, 'Adam': None, 'RMSprop': None, 'SGD': None}

early_stopping = EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

for i, _ in optimizers.items():

    print(f'\n{i}')
    model.compile(optimizer=i, loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    results = model.fit(train_x, train_df.Species.values, batch_size=4, epochs=10, callbacks=[early_stopping])

    best_accuracy = max(results.history['accuracy'])
    print(f'\nBest Accuracy: {best_accuracy:.4f} at epoch {results.history["accuracy"].index(best_accuracy) + 1}')
    optimizers[i] = round(best_accuracy, 4)

print('\n\nSUMMARY:')
print(optimizers)


Adadelta
Epoch 1/10
38/38 [==============================] - 1s 5ms/step - loss: 0.1013 - accuracy: 0.9400
Epoch 2/10
38/38 [==============================] - 0s 5ms/step - loss: 0.1006 - accuracy: 0.9400
Epoch 3/10
38/38 [==============================] - 0s 5ms/step - loss: 0.0999 - accuracy: 0.9400
Epoch 4/10
38/38 [==============================] - 0s 5ms/step - loss: 0.0993 - accuracy: 0.9467
Epoch 5/10
38/38 [==============================] - 0s 4ms/step - loss: 0.0987 - accuracy: 0.9467
Epoch 6/10
38/38 [==============================] - 0s 4ms/step - loss: 0.0981 - accuracy: 0.9467
Epoch 7/10
38/38 [==============================] - 0s 5ms/step - loss: 0.0975 - accuracy: 0.9467

Best Accuracy: 0.9467 at epoch 4

Adagrad
Epoch 1/10
38/38 [==============================] - 1s 5ms/step - loss: 0.0635 - accuracy: 0.9733
Epoch 2/10
38/38 [==============================] - 0s 5ms/step - loss: 0.0550 - accuracy: 0.9800
Epoch 3/10
38/38 [==============================] - 0s 4ms/step -

In [5]:
test_df = pd.read_csv('./data/iris.csv')
test_x = np.column_stack((test_df.SepalLengthCm.values, test_df.SepalWidthCm.values, test_df.PetalLengthCm.values, test_df.PetalWidthCm.values))

test_df['Species'] = test_df.Species.apply(lambda x: species_dict[x])

print("EVALUATION")
evaluation = model.evaluate(test_x, test_df.Species.values)

EVALUATION
5/5 [==============================] - 0s 3ms/step - loss: 0.1064 - accuracy: 0.9467


/home/hubert/.local/lib/python3.10/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


In [6]:
random_index = np.random.randint(0, len(test_df))

random_sample = test_x[random_index].reshape(1, -1)     # reshaping to (1, 4) as there are 4 features

ground_truth = test_df.iloc[random_index]['Species']
ground_truth_species = None


prediction_probs = model.predict(random_sample)
predicted_label = np.argmax(prediction_probs)
predicted_label_species = None

for key, value in species_dict.items():
    if value == ground_truth:
        ground_truth_species = key
    if value == predicted_label:
        predicted_label_species = key
    if ground_truth_species != None and predicted_label_species != None:
        break

# Print the ground truth and prediction
print(f"Ground Truth: {int(ground_truth)} ({ground_truth_species})")
print(f"Prediction: {predicted_label} ({predicted_label_species})\n(Probability distribution: {prediction_probs})")

1/1 [==============================] - 0s 123ms/step
Ground Truth: 2 (Iris-virginica)
Prediction: 2 (Iris-virginica)
(Probability distribution: [[0.00253047 0.8095319  0.99578637]])
